#### Optional: Run this cell to see available notebook commands ("magics").


In [14]:
import pandas as pd
from pyspark.sql import SparkSession
import boto3
from botocore.exceptions import ClientError
import json

In [3]:
# Inicialize a sessão Spark
spark = SparkSession.builder \
    .appName("ReadS3DataCreateTable") \
    .getOrCreate()

In [9]:
output_s3_path_delivery = "s3://218606466161-delivery/bancos_satisfacao"
df_bancos_satisfacao = spark.read.option("encoding", "UTF-8").parquet(output_s3_path_delivery)

In [15]:
secret_name = "rds!db-d64bc476-4474-40c8-b236-a8f5c68eb8fc"
region_name = "us-east-1"

# Create a Secrets Manager client
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

try:
    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )
except ClientError as e:
    # For a list of exceptions thrown, see
    # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    raise e

# Decrypts secret using the associated KMS key.
secret = get_secret_value_response['SecretString']
secrets = json.loads(secret)


In [34]:
jdbc_url = "jdbc:sqlserver://dbatividade.cvnvnfakh0k6.us-east-1.rds.amazonaws.com:1433"
tb = "dbatividade.dbo.banco_satisfacao"

connection_mssql_options = {
    "url": jdbc_url,
    "user": secrets["username"],
    "password": secrets["password"]}


df_bancos_satisfacao.write.mode("overwrite").jdbc(url=jdbc_url, \
                                                  table = tb, properties = connection_mssql_options)

In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [ ]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [ ]:
dyf = glueContext.create_dynamic_frame.from_catalog(database='database_name', table_name='table_name')
dyf.printSchema()

#### Example: Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [ ]:
df = dyf.toDF()
df.show()

#### Example: Visualize data with matplotlib


In [ ]:
import matplotlib.pyplot as plt

# Set X-axis and Y-axis values
x = [5, 2, 8, 4, 9]
y = [10, 4, 8, 5, 2]
  
# Create a bar chart 
plt.bar(x, y)
  
# Show the plot
%matplot plt

#### Example: Write the data in the DynamicFrame to a location in Amazon S3 and a table for it in the AWS Glue Data Catalog


In [ ]:
s3output = glueContext.getSink(
  path="s3://bucket_name/folder_name",
  connection_type="s3",
  updateBehavior="UPDATE_IN_DATABASE",
  partitionKeys=[],
  compression="snappy",
  enableUpdateCatalog=True,
  transformation_ctx="s3output",
)
s3output.setCatalogInfo(
  catalogDatabase="demo", catalogTableName="populations"
)
s3output.setFormat("glueparquet")
s3output.writeFrame(DyF)